In [2]:
## Only use this for Google Colab
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('drive/MyDrive/oldAIboy/houseprice')

!pip install category_encoders
!pip install optuna

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 10.8 MB/s eta 0:00:00


### XGBoost regressor

#### Setup modules

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import uniform, randint, loguniform

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from xgboost import XGBClassifier, DMatrix, XGBRegressor
from category_encoders import TargetEncoder
import pickle

import sys
print(sys.executable)

def data_preprocessing(traindataloc = "data/train.csv", testdataloc = "data/test.csv"):
  ## read in the data
  traindata = pd.read_csv(traindataloc)
  print("Full train dataset shape is {}".format(traindata.shape))
  testdata = pd.read_csv(testdataloc)
  print("Full test dataset shape is {}".format(testdata.shape))

  ##separate X and y: create X_train_combo and ylog (X_train_combo will be split into X_train and X_val later)
  X_mlb = traindata.drop('SalePrice', axis=1)
  X_mlb.set_index('Id', inplace=True)
  ylog = np.log(traindata['SalePrice']) ## apply log to target based on competition host's request
  ## set up the dataset X_test for final submission
  X_test = testdata.copy(deep=True) ##create a data copy
  X_test.set_index('Id', inplace=True)

  ylog.index=X_mlb.index

  ## feature cleaning
  ## drop 2 categorical features with super high NA%
  X_mlb.drop(['PoolQC','MiscFeature'], axis=1, inplace=True)
  ### fill NA for a bunch of categorical features - fill with 'None'
  catna_none = ['Alley','Fence','MasVnrType', 'FireplaceQu', 'GarageType', 'GarageQual', 'GarageCond', 'GarageFinish', 'BsmtFinType2', 'BsmtExposure', 'BsmtFinType1', 'BsmtQual', 'BsmtCond']
  X_mlb[catna_none] = X_mlb[catna_none].fillna('None')
  ### fill na for cat features 'Electrical' - fill with most frequent value
  most_frequent_value = X_mlb['Electrical'].mode()[0]
  X_mlb['Electrical'] = X_mlb['Electrical'].fillna(most_frequent_value)
  ### fill NA for 2 numerical features
  X_mlb['GarageYrBlt'] = X_mlb['GarageYrBlt'].fillna(1700) ##fill na of garageyrblt with 1700
  X_mlb['MasVnrArea'] = X_mlb['MasVnrArea'].fillna(0) ##fill na of MasVnrArea with 0
  ##fill na of LotFrontage with mean
  X_mlb['LotFrontage'] = X_mlb['LotFrontage'].fillna(X_mlb['LotFrontage'].mean())
  print("Full train dataset shape after cleaning is {}".format(X_mlb.shape))
  print("Full test dataset shape after cleaning is {}".format(X_test.shape))
  return X_mlb, ylog, X_test

def mean_encoding(X_mlb, ylog):
  # Split the data into training and validation sets with fixed validation size of 200 samples
  X_train, X_valid, y_train, y_valid = train_test_split(X_mlb, ylog, test_size=200, random_state=42)

  # Get list of categorical columns from Xcombo
  catcollist = X_mlb.select_dtypes(include=['object']).columns.tolist()

  # Apply mean encoding on the training set
  mean_encoder = TargetEncoder(cols=catcollist)
  X_train_encoded = mean_encoder.fit_transform(X_train, y_train)
  # Transform the test set using the fitted mean encoder
  X_valid_encoded = mean_encoder.transform(X_valid)

  print("Encoded Training Set shape: ", X_train_encoded.shape)
  print("Encoded Valid Set shape: ", X_valid_encoded.shape)
  return mean_encoder,X_train_encoded, X_valid_encoded, y_train, y_valid

def feature_selection(X_train_encoded, X_valid_encoded, y_train):
  # Initialize the model
  xgbmodel = XGBRegressor(objective='reg:squarederror',
                          n_estimators=100,
                          learning_rate=0.1,
                          eval_metric='rmse',
                          n_jobs=-1)

  # Perform RFE
  selector = RFE(estimator=xgbmodel, n_features_to_select=60, step=1)
  selector = selector.fit(X_train_encoded, y_train)

  # Get the selected features
  selected_features = X_train_encoded.columns[selector.support_]
  X_train_RFE = selector.transform(X_train_encoded)
  X_valid_RFE = selector.transform(X_valid_encoded)

  print("RFE Training Set shape: ", X_train_RFE.shape)
  print("RFE Valid Set shape: ", X_valid_RFE.shape)
  return selector, selected_features,X_train_RFE, X_valid_RFE

def model_training(model, param_dist, n_iter, cv, scoring, X_train_RFE, y_train, X_valid_RFE, y_valid):
  # Initialize RandomizedSearchCV
  random_search_rfe = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=n_iter, cv=cv, scoring=scoring, random_state=42, verbose=2, n_jobs=3)
  # Fit the model
  random_search_rfe.fit(X_train_RFE, y_train, eval_set=[(X_valid_RFE, y_valid)],verbose=False)
  # Print the best score
  print("Best score: ", np.sqrt((-random_search_rfe.best_score_)))
  # Print the best parameters
  print("Best parameters found: ", random_search_rfe.best_params_)
  # Use the best model to make predictions
  tunedmodel_rfe = random_search_rfe.best_estimator_
  # Evaluate the model using RMSE
  y_pred = tunedmodel_rfe.predict(X_valid_RFE)
  rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
  print(f'Root Mean Squared Error for validset: {rmse:.4f}')
  ytrain_pred = tunedmodel_rfe.predict(X_train_RFE)
  rmse_train = np.sqrt(mean_squared_error(y_train, ytrain_pred))
  print(f'Root Mean Squared Error for trainset: {rmse_train:.4f}')
  return random_search_rfe.best_params_

def test_data_cleaning(X_test, mean_encoder, selector):
  ## drop 2 categorical features with super high NA%
  ## drop 2 categorical features with super high NA%
  X_test2 = X_test.copy(deep=True)
  X_test2.drop(['PoolQC','MiscFeature'], axis=1, inplace=True)
  ### fill NA for a bunch of categorical features - fill with 'None'
  catna_none = ['Alley','Fence','MasVnrType', 'FireplaceQu', 'GarageType', 'GarageQual', 'GarageCond', 'GarageFinish', 'BsmtFinType2', 'BsmtExposure', 'BsmtFinType1', 'BsmtQual', 'BsmtCond']
  X_test2[catna_none] = X_test2[catna_none].fillna('None')
  ### fill na for cat features 'Electrical' - fill with most frequent value
  most_frequent_value = X_test2['Electrical'].mode()[0]
  X_test2['Electrical'] = X_test2['Electrical'].fillna(most_frequent_value)
  ### fill NA for 2 numerical features
  X_test2['GarageYrBlt'] = X_test2['GarageYrBlt'].fillna(1700) ##fill na of garageyrblt with 1700
  X_test2['MasVnrArea'] = X_test2['MasVnrArea'].fillna(0) ##fill na of MasVnrArea with 0
  ##fill na of LotFrontage with mean
  X_test2['LotFrontage'] = X_test2['LotFrontage'].fillna(X_test2['LotFrontage'].mean())

  ## implement mean-encoding
  X_test_encoded = mean_encoder.transform(X_test2)
  print("Training Set shape: ", X_train_encoded.shape)
  print("Valid Set shape: ", X_valid_encoded.shape)
  print("Test Set shape: ", X_test_encoded.shape)
  ##implement RFE selection
  X_test_RFE = selector.transform(X_test_encoded)
  print(X_test_RFE.shape)
  return X_test_RFE

def model_inference(X_test, X_test_RFE, selected_features, X_train_RFE, X_valid_RFE, y_train, y_valid, best_params):
  ## combine the train data and valid data
  X_train_RFEdf = pd.DataFrame(X_train_RFE, columns=selected_features)
  X_valid_RFEdf = pd.DataFrame(X_valid_RFE, columns=selected_features)
  X_trainrfe_combo = pd.concat([X_train_RFEdf, X_valid_RFEdf], axis=0)
  ## combine the train data and valid data
  y_train_RFE = pd.DataFrame(y_train)
  y_valid_RFE = pd.DataFrame(y_valid)
  y_trainrfe_combo = pd.concat([y_train_RFE, y_valid_RFE], axis=0, ignore_index=True)

  # Retrieve the best parameters from RandomizedSearchCV
  additional_params = {
      'objective':'reg:squarederror',
      'eval_metric':'rmse'
  }
  # Update the best parameters with the new ones
  best_params.update(additional_params)
  # Create a new model with the updated parameters
  model_inference = XGBRegressor(**best_params)
  # Train the final model on the combined dataset (or any dataset you choose)
  model_inference.fit(X_trainrfe_combo, y_trainrfe_combo)
  ytrain_pred_rfe = model_inference.predict(X_trainrfe_combo)
  rmse_train = np.sqrt(mean_squared_error(y_trainrfe_combo, ytrain_pred_rfe))
  print(f'Root Mean Squared Error for trainset: {rmse_train:.4f}')

  y_pred_test = model_inference.predict(X_test_RFE)
  testpred = np.exp(y_pred_test)
  print(f'average sales prediction: {np.mean(testpred)}')

  test_id=X_test.index.tolist()
  sub = pd.DataFrame()
  sub['Id'] = test_id
  sub['SalePrice'] = testpred
  print(f'snapshot of inference results: {sub.head()}')
  return sub

/usr/bin/python3


#### Implement XGBregressor

In [4]:
%%time
## data preprocessing, feature enginnering, feature selection
X_mlb, ylog, X_test = data_preprocessing()
mean_encoder, X_train_encoded, X_valid_encoded, y_train, y_valid = mean_encoding(X_mlb, ylog)
selector, selected_features, X_train_RFE, X_valid_RFE = feature_selection(X_train_encoded, X_valid_encoded, y_train)

Full train dataset shape is (1460, 81)
Full test dataset shape is (1459, 80)
Full train dataset shape after cleaning is (1460, 77)
Full test dataset shape after cleaning is (1459, 79)
Encoded Training Set shape:  (1260, 77)
Encoded Valid Set shape:  (200, 77)
RFE Training Set shape:  (1260, 60)
RFE Valid Set shape:  (200, 60)
CPU times: user 55.3 s, sys: 5.43 ms, total: 55.3 s
Wall time: 6.7 s


In [17]:
%%time
## data preprocessing, feature enginnering, feature selection
X_mlb, ylog, X_test = data_preprocessing()
mean_encoder, X_train_encoded, X_valid_encoded, y_train, y_valid = mean_encoding(X_mlb, ylog)
selector, selected_features, X_train_RFE, X_valid_RFE = feature_selection(X_train_encoded, X_valid_encoded, y_train)

## Model training with parameter tuning
model = XGBRegressor(objective='reg:squarederror',eval_metric='rmse')
n_iter=50
cv=3
scoring='neg_mean_squared_error'
# Define the parameter distribution
param_dist = { 'n_estimators': randint(150, 500)
                ,'max_depth': randint(3, 10)  # Randomly pick between 3 and 10
                ,'learning_rate': loguniform(0.005, 0.3)  # Log-uniform distribution between 0.01 and 0.3
                ,'subsample': uniform(0.5, 0.5)  # Randomly pick between 0.6 and 1.0
                ,'colsample_bytree': uniform(0.4, 0.6)  # Randomly pick between 0.6 and 1.0
                ,'gamma': uniform(0, 0.5)  # Randomly pick between 0 and 0.5
                ,'min_child_weight': randint(1, 6)  # Randomly pick between 1 and 5
                #,'reg_alpha': uniform(0, 10)  # Randomly pick between 0 and 10
                #,'reg_lambda': uniform(0, 10)  # Randomly pick between 0 and 10
             }

best_params = model_training(model, param_dist, n_iter, cv, scoring, X_train_RFE, y_train, X_valid_RFE, y_valid)

## Model Inference
X_test_RFE = test_data_cleaning(X_test, mean_encoder, selector)
pred_sub=model_inference(X_test, X_test_RFE, selected_features, X_train_RFE, X_valid_RFE, y_train, y_valid, best_params)

Full train dataset shape is (1460, 81)
Full test dataset shape is (1459, 80)
Full train dataset shape after cleaning is (1460, 77)
Full test dataset shape after cleaning is (1459, 79)
Encoded Training Set shape:  (1260, 77)
Encoded Valid Set shape:  (200, 77)
RFE Training Set shape:  (1260, 60)
RFE Valid Set shape:  (200, 60)
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best score:  0.12653205175949037
Best parameters found:  {'colsample_bytree': np.float64(0.916438349953806), 'gamma': np.float64(0.0034760652655953517), 'learning_rate': np.float64(0.04047212046195181), 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 374, 'subsample': np.float64(0.5599326836668415)}
Root Mean Squared Error for validset: 0.1270
Root Mean Squared Error for trainset: 0.0753
Training Set shape:  (1260, 77)
Valid Set shape:  (200, 77)
Test Set shape:  (1459, 77)
(1459, 60)
Root Mean Squared Error for trainset: 0.0794
average sales prediction: 177583.6875
snapshot of inference results:

### Deep Neutral Network
#### DNN is famous for creating new features and handling feature selection, let's give it a try!

#### Some preparation - import library, build pytorch data loader

In [8]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import optuna
import pickle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 32         # Batch size

# ====== Data Pipeline - normalize and pack train/valid data to Loaders ======
def dnn_loader (batch_size, X_train_encoded, X_valid_encoded, y_train, y_valid):
  ## give the pre-RFE dataset to DNN to let it implement feature engineering
  train_features_np = X_train_encoded.values
  valid_features_np = X_valid_encoded.values

  # Normalize features using StandardScaler
  scaler = StandardScaler()
  train_features_np = scaler.fit_transform(train_features_np)
  valid_features_np = scaler.transform(valid_features_np)

  ## Normalize prediction target
  target_scaler = StandardScaler()
  y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1))
  y_valid_scaled = target_scaler.transform(y_valid.values.reshape(-1, 1))

  # Convert NumPy arrays to PyTorch tensors after normalization
  train_features = torch.tensor(train_features_np, dtype=torch.float32).to(device)
  valid_features = torch.tensor(valid_features_np, dtype=torch.float32).to(device)
  train_labels = torch.tensor(y_train_scaled, dtype=torch.float32).view(-1, 1).to(device)
  valid_labels = torch.tensor(y_valid_scaled, dtype=torch.float32).view(-1, 1).to(device)

  # Create dataloaders
  train_dataset = TensorDataset(train_features, train_labels)
  valid_dataset = TensorDataset(valid_features, valid_labels)
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
  return train_loader, val_loader, valid_features, valid_labels, scaler, target_scaler

train_loader, val_loader, valid_features, valid_labels, scaler, target_scaler = dnn_loader(batch_size, X_train_encoded, X_valid_encoded, y_train, y_valid)

#### DNN tuning and training - obtained RMSE ON VALIDSET 0.1243

In [6]:
# Fixed hyperparameters (for aspects not tuned by Optuna)
dropout_rate = 0.2      # Dropout rate in the MLP
max_epochs = 100        # Maximum number of epochs per trial
patience = 10           # Patience for early stopping (used in scheduler)
criterion = nn.MSELoss()  # Mean Squared Error for regression

# ====== Model Definition Functions ======
def define_model(trial):
    """Define the MLP architecture for a given Optuna trial.
       Tuning both number of hidden layers and neurons per layer."""
    n_layers = trial.suggest_int("n_layers", 2, 4)
    layers = []
    input_dim = X_train_encoded.shape[1]  # number of features
    for i in range(n_layers):
        units = trial.suggest_int(f"n_units_l{i}", 64, 256)
        layers.append(nn.Linear(input_dim, units))
        layers.append(nn.ReLU())
        if dropout_rate > 0:
            layers.append(nn.Dropout(dropout_rate))
        input_dim = units
    layers.append(nn.Linear(input_dim, 1))  # Output layer for regression
    model = nn.Sequential(*layers)
    return model

def build_model_from_config(config):
    """Rebuild a model from a configuration dictionary (e.g., best found params)."""
    n_layers = config["n_layers"]
    layers = []
    input_dim = X_train_encoded.shape[1]
    for i in range(n_layers):
        units = config[f"n_units_l{i}"]
        layers.append(nn.Linear(input_dim, units))
        layers.append(nn.ReLU())
        if dropout_rate > 0:
            layers.append(nn.Dropout(dropout_rate))
        input_dim = units
    layers.append(nn.Linear(input_dim, 1))
    model = nn.Sequential(*layers)
    return model

## generate the data loaders for DNN model training
# train_loader, val_loader, valid_features, valid_labels, target_scaler = dnn_loader(batch_size, X_train_encoded, X_valid_encoded, y_train, y_valid)

# ====== Training Function with ReduceLROnPlateau ======
def train_model(model, lr):
    """
    Train the model for up to max_epochs with early stopping using
    ReduceLROnPlateau for dynamic learning rate adjustment.
    Returns the best validation loss and the best model state dict.
    """
    # Create optimizer with the tuned learning rate
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    # Setup ReduceLROnPlateau scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', patience=patience, factor=0.5, min_lr=1e-6, verbose=True
    )

    best_val_loss = float('inf')
    best_state_dict = None

    for epoch in range(1, max_epochs + 1):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluate on validation set
        model.eval()
        total_samples = 0
        with torch.no_grad():
            preds = model(valid_features)
            val_loss = F.mse_loss(preds, valid_labels).item()

        # Update best validation loss and state dict if improved
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_state_dict = {k: v.cpu() for k, v in model.state_dict().items()}

        # Step the scheduler using the validation loss
        scheduler.step(val_loss)
        current_lr = optimizer.param_groups[0]['lr']
        if epoch % 100 == 0:
          print(f"Epoch {epoch}: val_loss={val_loss:.6f}, lr={current_lr:.1e}")

        # Early stopping: if lr has reached the minimum threshold, stop training
        if current_lr <= scheduler.min_lrs[0]:
            print(f"Learning rate reached minimum threshold ({scheduler.min_lrs[0]}). Early stopping.")
            break

    if best_state_dict is not None:
        model.load_state_dict(best_state_dict)
    return best_val_loss, best_state_dict

# ====== Global Tracking for Best Model ======
best_global_model_state = None
best_global_val_loss = float('inf')
best_global_params = None

# ====== Optuna Objective Function ======
def objective(trial):
    global best_global_model_state, best_global_val_loss, best_global_params

    # Tune the initial learning rate as well as the network architecture
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    model = define_model(trial).to(device)
    val_loss, best_state = train_model(model, lr)

    # Update global best if current trial is better
    if val_loss < best_global_val_loss:
        best_global_val_loss = val_loss
        best_global_params = trial.params  # includes architecture & learning rate
        best_global_model_state = best_state
        torch.save(best_global_model_state, "best_dnnmodel_statedict_0317.pth")
        print(f"*** New best model found! Val MSE={val_loss:.6f}, Params={best_global_params}")
    return val_loss  # Optuna minimizes this

In [9]:
%%time
# ====== Run Optuna Study ======
study = optuna.create_study(direction="minimize")
print("Starting hyperparameter search...")
study.optimize(objective, n_trials=100, gc_after_trial=True)  # 100 trials

# ====== Hyperparameter Tuning Results ======
print("\nOptuna search completed.")
print(f"Number of finished trials: {len(study.trials)}")
print(f"Best trial index: {study.best_trial.number}")
print(f"Best validation MSE: {study.best_value:.6f}")
best_params = study.best_params
print("Best hyperparameter configuration:", best_params)
best_val_rmse = study.best_value ** 0.5
print(f"Best validation RMSE: {best_val_rmse:.6f}")

# Save the best parameters to a file
with open('dnn_best_params_0317.pkl', 'wb') as file:
    pickle.dump(best_params, file)


# rebuild the DNN
with open('dnn_best_params_0317.pkl', 'rb') as file:
    best_params = pickle.load(file)
best_model = build_model_from_config(best_params).to(device)
best_model.load_state_dict(torch.load("best_dnnmodel_statedict_0317.pth"))

# Evaluation on validation set
best_model.eval()
with torch.no_grad():
    valid_predictions = best_model(valid_features)
    valid_predictions = valid_predictions.cpu().numpy()  # if using GPU
    valid_predictions_inv = target_scaler.inverse_transform(valid_predictions)
    rmse = np.sqrt(mean_squared_error(y_valid, valid_predictions_inv))
    print(f"RMSE on validation set: {rmse:.4f}")

[I 2025-03-18 01:24:24,968] A new study created in memory with name: no-name-21771f26-26d1-4330-9f51-19189b829227


Starting hyperparameter search...


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 100: val_loss=0.129977, lr=1.3e-05


[I 2025-03-18 01:24:37,544] Trial 0 finished with value: 0.11548255383968353 and parameters: {'lr': 0.0033178263844343987, 'n_layers': 2, 'n_units_l0': 209, 'n_units_l1': 79}. Best is trial 0 with value: 0.11548255383968353.


...

[I 2025-03-18 01:37:09,677] Trial 98 finished with value: 0.11680358648300171 and parameters: {'lr': 0.00033007256660124454, 'n_layers': 2, 'n_units_l0': 232, 'n_units_l1': 108}. Best is trial 7 with value: 0.102622389793396.


Epoch 100: val_loss=0.126168, lr=1.3e-06


[I 2025-03-18 01:37:16,999] Trial 99 finished with value: 0.11158479750156403 and parameters: {'lr': 5.4820096395292546e-05, 'n_layers': 2, 'n_units_l0': 253, 'n_units_l1': 220}. Best is trial 7 with value: 0.102622389793396.


Epoch 100: val_loss=0.111854, lr=3.4e-06

Optuna search completed.
Number of finished trials: 100
Best trial index: 7
Best validation MSE: 0.102622
Best hyperparameter configuration: {'lr': 0.00025924210512089416, 'n_layers': 2, 'n_units_l0': 189, 'n_units_l1': 237}
Best validation RMSE: 0.320347
RMSE on validation set: 0.1253
CPU times: user 12min 33s, sys: 11.4 s, total: 12min 44s
Wall time: 12min 52s


<timed exec>:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


### Conduct final inference - RMSE returned by kaggle: 0.1286

#### refit the model based on best parameter and combined train/valid set


In [13]:
### refit the DNN based on the combined train/valid dataset
def dnn_refit():
  # ===== Combine Train & Validation Data =====
  X_train_valid_np = np.vstack((X_train_encoded, X_valid_encoded))
  y_train_valid_np = np.hstack((y_train, y_valid))

  # Standardize features based on previously fitted scaler
  X_train_valid_scaled = scaler.transform(X_train_valid_np)
  y_train_valid_scaled = target_scaler.transform(y_train_valid_np.reshape(-1, 1))

  # Convert to torch tensors
  X_train_valid = torch.tensor(X_train_valid_scaled, dtype=torch.float32).to(device)
  y_train_valid = torch.tensor(y_train_valid_scaled, dtype=torch.float32).view(-1, 1).to(device)

  # Create dataset & dataloader
  train_valid_dataset = TensorDataset(X_train_valid, y_train_valid)
  train_valid_loader = DataLoader(train_valid_dataset, batch_size=batch_size, shuffle=True)

  # ===== Load Best Hyperparameters from Optuna =====
  with open('dnn_best_params_0317.pkl', 'rb') as file:
      best_params = pickle.load(file)
  print("Refitting model with best hyperparameters:", best_params)

  # Instantiate the model
  modelrefit = build_model_from_config(best_params).to(device)

  # Define loss function and optimizer
  criterion = nn.MSELoss()
  optimizer = torch.optim.Adam(modelrefit.parameters(), lr=best_params["lr"])

  # ===== Train the Model with Best Hyperparameters =====
  num_epochs = 100  # You can adjust based on validation performance
  for epoch in range(num_epochs):
      modelrefit.train()
      epoch_loss = 0.0
      for inputs, targets in train_valid_loader:
          optimizer.zero_grad()
          outputs = modelrefit(inputs)
          loss = criterion(outputs, targets)
          loss.backward()
          optimizer.step()
          epoch_loss += loss.item()

      if (epoch + 1) % 10 == 0:  # Print loss every 10 epochs
          print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_valid_loader):.4f}')

  # # ===== Save the Model for Future Inference =====
  # torch.save(model.state_dict(), "dnnmodel2_inference.pth")
  # print("Final trained model saved as 'dnnmodel2_inference.pth'")
  return modelrefit

### process testdata
def testdata_processing():
## drop 2 categorical features with super high NA%
  X_test2 = X_test.copy(deep=True)
  X_test2.drop(['PoolQC','MiscFeature'], axis=1, inplace=True)
  ### fill NA for a bunch of categorical features - fill with 'None'
  catna_none = ['Alley','Fence','MasVnrType', 'FireplaceQu', 'GarageType', 'GarageQual', 'GarageCond', 'GarageFinish', 'BsmtFinType2', 'BsmtExposure', 'BsmtFinType1', 'BsmtQual', 'BsmtCond']
  X_test2[catna_none] = X_test2[catna_none].fillna('None')
  ### fill na for cat features 'Electrical' - fill with most frequent value
  most_frequent_value = X_test2['Electrical'].mode()[0]
  X_test2['Electrical'] = X_test2['Electrical'].fillna(most_frequent_value)
  ### fill NA for 2 numerical features
  X_test2['GarageYrBlt'] = X_test2['GarageYrBlt'].fillna(1700) ##fill na of garageyrblt with 1700
  X_test2['MasVnrArea'] = X_test2['MasVnrArea'].fillna(0) ##fill na of MasVnrArea with 0
  ##fill na of LotFrontage with mean
  X_test2['LotFrontage'] = X_test2['LotFrontage'].fillna(X_test2['LotFrontage'].mean())
  ## implement meanencoder
  X_test_encoded = mean_encoder.transform(X_test2)

  print("Training Set shape: ", X_train_encoded.shape)
  print("Valid Set shape: ", X_valid_encoded.shape)
  print("Test Set shape: ", X_test_encoded.shape)
  ## handle NAs only existing in testdata
  X_test_encoded['BsmtFinSF1'] = X_test_encoded['BsmtFinSF1'].fillna(X_test_encoded['BsmtFinSF1'].mean())
  X_test_encoded['BsmtFinSF2'] = X_test_encoded['BsmtFinSF2'].fillna(X_test_encoded['BsmtFinSF2'].mean())
  X_test_encoded['BsmtUnfSF'] = X_test_encoded['BsmtUnfSF'].fillna(0)
  X_test_encoded['TotalBsmtSF'] = X_test_encoded['TotalBsmtSF'].fillna(X_test_encoded['TotalBsmtSF'].mean())
  X_test_encoded['BsmtFullBath'] = X_test_encoded['BsmtFullBath'].fillna(X_test_encoded['BsmtFullBath'].median())
  X_test_encoded['BsmtHalfBath'] = X_test_encoded['BsmtHalfBath'].fillna(X_test_encoded['BsmtHalfBath'].median())
  X_test_encoded['GarageCars'] = X_test_encoded['GarageCars'].fillna(X_test_encoded['GarageCars'].median())
  X_test_encoded['GarageArea'] = X_test_encoded['GarageArea'].fillna(X_test_encoded['GarageArea'].mean())
  ## double check the result
  print(f'double check NA vavlues: {X_test_encoded.isnull().sum().sum()}')
  ### standardize inference features
  test_features_np = X_test_encoded.values
  test_features_np = scaler.transform(test_features_np)
  test_features = torch.tensor(test_features_np, dtype=torch.float32).to(device)
  return test_features




In [14]:
modelrefit = dnn_refit()
test_features = testdata_processing()

# Load the model for inference
#model = build_model_from_config(best_params).to(device)
#model.load_state_dict(torch.load("dnnmodel2_inference.pth"))

modelrefit.eval()  # Set to evaluation mode
with torch.no_grad():
    test_predictions = modelrefit(test_features)
    test_predictions = test_predictions.cpu().numpy()  # if using GPU
    test_predictions_inv = target_scaler.inverse_transform(test_predictions)

testpred_dnn = np.exp(test_predictions_inv)
print(np.mean(testpred_dnn))

test_id=X_test.index.tolist()
def submit_method(test_id, y_prediction):
    sub = pd.DataFrame()
    sub['Id'] = test_id
    sub['SalePrice'] = y_prediction
    print(sub.shape)
    return sub

subcsv=submit_method(test_id, testpred_dnn)



Refitting model with best hyperparameters: {'lr': 0.00025924210512089416, 'n_layers': 2, 'n_units_l0': 189, 'n_units_l1': 237}
Epoch [10/100], Loss: 0.0993
Epoch [20/100], Loss: 0.0671
Epoch [30/100], Loss: 0.0551
Epoch [40/100], Loss: 0.0488
Epoch [50/100], Loss: 0.0403
Epoch [60/100], Loss: 0.0374
Epoch [70/100], Loss: 0.0336
Epoch [80/100], Loss: 0.0309
Epoch [90/100], Loss: 0.0299
Epoch [100/100], Loss: 0.0269
Training Set shape:  (1260, 77)
Valid Set shape:  (200, 77)
Test Set shape:  (1459, 77)
double check NA vavlues: 0
178846.3
(1459, 2)
